In [13]:
# packages
import os
import numpy as np
import pandas as pd

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense

from sklearn.metrics import classification_report

In [9]:
# mount your google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
df = pd.read_csv('/content/drive/My Drive/ms_wind_curtailment_prediction/curtailment_target_features.csv', sep = ';', index_col=0)

In [11]:
# Split the entire dataset into training and test sets
cutoff_time = "2023-01-01"
train = df[df.index < cutoff_time]
test = df[df.index >= cutoff_time]
X_train = train.drop('redispatch', axis = 1)
y_train = train['redispatch']
X_test = test.drop('redispatch', axis = 1)
y_test = test['redispatch']

In [15]:
# preprocessing pipelines
preprocessor = Pipeline([
    ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
    ('scaler', StandardScaler())
])

smote = SMOTE(random_state=13)

X_train_preprocessed = preprocessor.fit_transform(X_train)
X_train_preprocessed, y_train_preprocessed = smote.fit_resample(X_train_preprocessed, y_train)
X_test_preprocessed = preprocessor.transform(X_test)

In [18]:
# define model
def dl_bin_class_model():
    model = tf.keras.Sequential([
        Dense(64, activation='relu', input_dim=12),
        Dense(32, activation='relu'),
        Dense(1, activation='sigmoid')
    ])

# Define number of epochs and learning rate decay
N_TRAIN = len(X_train_preprocessed)
EPOCHS = 50
BATCH_SIZE = 32
STEPS_PER_EPOCH = N_TRAIN // BATCH_SIZE
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
    0.01,
    decay_steps=STEPS_PER_EPOCH,
    decay_rate=1,
    staircase=False)

# Define optimizer
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, name='Adam')

# Define model
model = dl_bin_class_model()

# Compile the model with custom optimizer and metrics
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy', tf.keras.metrics.Recall()])

# Train model
history = model.fit(X_train_preprocessed,
                    y_train_preprocessed,
                    validation_split=0.2,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    verbose=1)

Epoch 1/2000
4967/4967 [==============================] - 16s 3ms/step - loss: 0.3718 - accuracy: 0.8237 - recall: 0.7785 - val_loss: 0.3955 - val_accuracy: 0.8449 - val_recall: 0.8449
Epoch 2/2000
4967/4967 [==============================] - 15s 3ms/step - loss: 0.2491 - accuracy: 0.8942 - recall: 0.8861 - val_loss: 0.3445 - val_accuracy: 0.8705 - val_recall: 0.8705
Epoch 3/2000
4967/4967 [==============================] - 16s 3ms/step - loss: 0.1995 - accuracy: 0.9199 - recall: 0.9178 - val_loss: 0.1840 - val_accuracy: 0.9477 - val_recall: 0.9477
Epoch 4/2000
4967/4967 [==============================] - 16s 3ms/step - loss: 0.1750 - accuracy: 0.9310 - recall: 0.9305 - val_loss: 0.1321 - val_accuracy: 0.9619 - val_recall: 0.9619
Epoch 5/2000
4967/4967 [==============================] - 22s 4ms/step - loss: 0.1591 - accuracy: 0.9391 - recall: 0.9406 - val_loss: 0.1594 - val_accuracy: 0.9525 - val_recall: 0.9525
Epoch 6/2000
4967/4967 [==============================] - 20s 4ms/step - lo

KeyboardInterrupt: 

In [ ]:
'''# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test_preprocessed, y_test)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')'''


In [40]:
# Assuming you have already trained your model and obtained predictions on the test data
y_pred = model.predict(X_test_preprocessed)

# Print classification report
print(classification_report(y_test, y_pred))

ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [39]:
# Define your deep learning model
model = Sequential([
    Dense(64, activation='relu', input_dim=12),  # Example input_dim
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')  # Example output layer for binary classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['recall'])

# Train the model
model.fit(X_train_preprocessed, y_train_preprocessed, epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test_preprocessed, y_test)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

# Make predictions (optional)
predictions = model.predict(X_test_preprocessed)

Epoch 1/10
4967/4967 [==============================] - 15s 3ms/step - loss: 0.4285 - accuracy: 0.7931 - val_loss: 0.5099 - val_accuracy: 0.7659
Epoch 2/10
4967/4967 [==============================] - 12s 2ms/step - loss: 0.3371 - accuracy: 0.8475 - val_loss: 0.3326 - val_accuracy: 0.8813
Epoch 3/10
4967/4967 [==============================] - 13s 3ms/step - loss: 0.2945 - accuracy: 0.8728 - val_loss: 0.3802 - val_accuracy: 0.8529
Epoch 4/10
4967/4967 [==============================] - 10s 2ms/step - loss: 0.2676 - accuracy: 0.8873 - val_loss: 0.3962 - val_accuracy: 0.8365
Epoch 5/10
4967/4967 [==============================] - 9s 2ms/step - loss: 0.2486 - accuracy: 0.8967 - val_loss: 0.3867 - val_accuracy: 0.8401
Epoch 6/10
4967/4967 [==============================] - 11s 2ms/step - loss: 0.2346 - accuracy: 0.9041 - val_loss: 0.2775 - val_accuracy: 0.9042
Epoch 7/10
4967/4967 [==============================] - 13s 3ms/step - loss: 0.2228 - accuracy: 0.9107 - val_loss: 0.2123 - val_acc